### import the library we want to use

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

### Load Dataset

In [3]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5172,5173,15813095,Nwebube,553,France,Male,37,2,0.00,2,1,0,33877.29,0
8697,8698,15644551,Wimble,751,Spain,Female,37,3,99773.85,2,1,0,54865.92,0
2233,2234,15791268,Neumann,565,Spain,Male,38,0,122447.76,1,0,0,67339.34,0
5593,5594,15597968,Fyans,617,Spain,Male,50,7,0.00,1,1,0,184839.70,1
1284,1285,15742229,Mackay,583,France,Male,59,7,127450.14,1,0,1,67552.71,0


### Feature Enginnering Dataset

In [8]:
df.shape

(10000, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
# Drop column we doesnt use
df = df.drop(['RowNumber','CustomerId','Surname'], axis='columns')
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [9]:
for col in df:
  if df[col].dtypes == 'object':
    print(f'{col} : {df[col].unique()}')

Geography : ['France' 'Spain' 'Germany']
Gender : ['Female' 'Male']


### One Hot Column with have object

In [11]:
df = pd.get_dummies(df, columns=['Geography','Gender'])
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


### Scaling Columns with have int value

In [13]:
from sklearn.preprocessing import MinMaxScaler

col_int = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary'] 
scaler = MinMaxScaler()

df[col_int] = scaler.fit_transform(df[col_int])
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


### Build Model with Imbalanced Data 

In [15]:
from sklearn.model_selection import train_test_split

In [14]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [24]:
X = df.drop('Exited', axis='columns')
y = df['Exited']

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 13), (2000, 13), (8000,), (2000,))

In [33]:
def ANN(X_train, X_test, y_train, y_test, epochs, loss='binary_crossentropy'):
  model = keras.Sequential([
      keras.layers.Dense(13, input_dim=13, activation='relu'),
      keras.layers.Dense(10, activation='relu'),
      keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

  model.fit(X_train, y_train, epochs=epochs)

  print(model.evaluate(X_test, y_test))

  y_pred = model.predict(X_test)
  y_pred = np.round(y_pred)

  print(model.summary())

  print(classification_report(y_test, y_pred))

In [34]:
pred1 = ANN(X_train, X_test, y_train, y_test, 100, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5602 - accuracy: 0.7270
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4738 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4622 - accuracy: 0.7971
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4538 - accuracy: 0.8058
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4450 - accuracy: 0.8091
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4365 - accuracy: 0.8123
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4303 - accuracy: 0.8165
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4236 - accuracy: 0.8188
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4171 - accuracy: 0.8217
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4082 - accura

### Build Model wwith Balanced Data with SMOTE

In [35]:
smote = SMOTE(sampling_strategy='minority')
x_sm, y_sm = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(x_sm, y_sm, test_size=0.2, random_state=1, stratify=y_sm)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12740, 13), (3186, 13), (12740,), (3186,))

In [36]:
pred2 = ANN(X_train, X_test, y_train, y_test, 100, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6520 - accuracy: 0.6159
Epoch 2/100
399/399 [==============================] - 1s 3ms/step - loss: 0.6068 - accuracy: 0.6702
Epoch 3/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5790 - accuracy: 0.6969
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5591 - accuracy: 0.7148
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7282
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5247 - accuracy: 0.7392
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5100 - accuracy: 0.7495
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4980 - accuracy: 0.7582
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4866 - accuracy: 0.7624
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4780 - accura